<a href="https://colab.research.google.com/github/MichaelPaulukonis/notebooks/blob/main/Variant_of_Latent_Diffusion_Upscale.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#<font face="Trebuchet MS" size="6">Neural Image Super-Resolution<font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><font color="#999" size="4">Latent Diffusion upscale</font><font color="#999" size="4">&nbsp;&nbsp;&nbsp;|&nbsp;&nbsp;&nbsp;</font><a href="https://github.com/olaviinha/NeuralImageSuperResolution" target="_blank"><font color="#999" size="4">Github</font></a>

This notebook implements Superresolution Upscale from [Latent Diffusion](https://github.com/CompVis/latent-diffusion) in an attempt to improve and enhance image quality.

`input` may be a file path or a directory path. All paths should berelative to your Google Drive root. I.e. if your Google Drive has a directory called _images_ and under that directory you have a file _face.jpg_, then `input` value should be `images/face.jpg`

Images can be either jpeg or PNG, and the originals will be padded to multiples of 64.

In [ ]:

#@title #Setup
#@markdown This cell needs to be run only once. It will mount your Google Drive and setup prerequisites.<br>
#@markdown <small>Mounting Drive will enable this notebook to save outputs directly to your Drive. Otherwise you will need to copy/download them manually from this notebook.</small>

force_setup = False
repositories = [
  'https://github.com/olaviinha/latent-diffusion.git',
  'https://github.com/olaviinha/taming-transformers'
]
pip_packages = 'ipywidgets omegaconf>=2.0.0 pytorch-lightning>=1.0.8 torch-fidelity einops'
apt_packages = ''
mount_drive = True #@param {type:"boolean"}
skip_setup = False #@ param {type:"boolean"}

# Download the repo from Github
import os
from google.colab import output, files
import warnings
warnings.filterwarnings('ignore')
%cd /content/

# inhagcutils
if not os.path.isfile('/content/inhagcutils.ipynb') and force_setup == False:
  !pip -q install import-ipynb {pip_packages}
  if apt_packages != '':
    !apt-get update && apt-get install {apt_packages}
  !curl -s -O https://raw.githubusercontent.com/olaviinha/inhagcutils/master/inhagcutils.ipynb
import import_ipynb
from inhagcutils import *

# Mount Drive
if mount_drive == True:
  if not os.path.isdir('/content/drive'):
    from google.colab import drive
    drive.mount('/content/drive')
    drive_root = '/content/drive/My Drive'
  if not os.path.isdir('/content/mydrive'):
    os.symlink('/content/drive/My Drive', '/content/mydrive')
    drive_root = '/content/mydrive/'
  drive_root_set = True
else:
  create_dirs(['/content/faux_drive'])
  drive_root = '/content/faux_drive/'

if len(repositories) > 0 and skip_setup == False:
  for repo in repositories:
    %cd /content/
    install_dir = fix_path('/content/'+path_leaf(repo).replace('.git', ''))
    repo = repo if '.git' in repo else repo+'.git'
    !git clone {repo}
    if os.path.isfile(install_dir+'setup.py') or os.path.isfile(install_dir+'setup.cfg'):
      !pip install -e {install_dir}
    if os.path.isfile(install_dir+'requirements.txt'):
      !pip install -r {install_dir}/requirements.txt

if len(repositories) == 1:
  %cd {install_dir}

dir_tmp = '/content/tmp/'
dir_tmp_in = '/content/tmp_in/'
dir_tmp_out = '/content/tmp_out/'
create_dirs([dir_tmp, dir_tmp_in, dir_tmp_out])

import time, sys
from datetime import timedelta
import math



#-- start --


sys.path.append(".")
sys.path.append('./taming-transformers')
from taming.models import vqgan # checking correct import from taming

%cd latent-diffusion
import ipywidgets as widgets
from IPython.display import display

mode = widgets.Select(options=['superresolution'], value='superresolution', description='Task:')
#display(mode)

from notebook_helpers import get_model
model = get_model(mode.value)

from notebook_helpers import run
import torch
import numpy as np
from PIL import Image

#-- end --
output.clear()
# !nvidia-smi
op(c.ok, 'Setup finished.')

In [ ]:
#@title # Do stuff
# from drive root (don't include root)
# (height and width may have to be multiples of 64 or cropping will occur)
input = "Art general/to_upscale" #@param {type:"string"}
output_dir = "Art general/upscaled" #@param {type:"string"}
steps = 50 #@param {type:"integer"}

uniq_id = gen_id()
output.clear()

# TODO: optionally log all of this (in/out/resize, time of each, etc)

if os.path.isfile(drive_root+input):
  # TODO: pad image to size
  inputs = [drive_root+input]
  dir_in = path_dir(drive_root+input)
elif os.path.isdir(drive_root+input):
  dir_in = drive_root+fix_path(input)
  # What to do if input is directory path
  inputs = list_images(dir_in)
elif os.path.isdir(drive_root+input) and '*' in input:
  dir_in = path_dir(drive_root+input)
  inputs = glob(drive_root+input)
else:
  op(c.fail, 'FAIL!', 'Input should be a path to an image file or a directory of image files.')
  sys.exit('Input not understood.')

# Output
if output_dir == '':
  dir_out = dir_in
else:
  if not os.path.isdir(drive_root+output_dir):
    os.mkdir(drive_root+output_dir)
  dir_out = drive_root+fix_path(output_dir)

total = len(inputs)
timer_start = time.time()

def lcm(x, y):
    x2 = math.ceil(x/64)*64
    y2 = math.ceil(y/64)*64
    return x2, y2


def add_margin(pil_img):
    width, height = pil_img.size
    new_width, new_height = lcm(width, height)
    if pil_img.mode == 'PNG':
        padded_image = Image.new('RGBA', (new_width, new_height), (0, 0, 0, 0))
    else:
        padded_image = Image.new('RGB', (new_width, new_height), (0, 0, 0))
    # result = Image.new(pil_img.mode, (new_width, new_height), color)
    padded_image.paste(pil_img, (0, 0))

    op(c.ok, f"original size: {width}, {height}")
    op(c.ok, f"new size: {new_width}, {new_height}")

    return padded_image

# -- DO THINGS --
for i, input in enumerate(inputs, 1):
  ndx = str(i)+'/'+str(total)+' '
  op(c.title, ndx+'Processing', input.replace(drive_root, ''), time=True)

  # NOTE: overwrites original image w/ padded version
  # TODO: improve on this?
  # TODO: remove padding after processing?
  # TODO: .png have to be handled differently

    # Create a new image with the padded dimensions and a transparent background
    # padded_image = Image.new('RGBA', (new_width, new_height), (0, 0, 0, 0))

    # Paste the original image onto the padded image with the specified padding
    # padded_image.paste(image, (padding, padding))

  im = Image.open(input)
  im_new = add_margin(im)
  im_new.save(input)

  img_out = dir_out+uniq_id+'_'+str(steps)+'steps_'+path_leaf(input)
  logs = run(model["model"], input, mode.value, steps)

  sample = logs["sample"]
  sample = sample.detach().cpu()
  sample = torch.clamp(sample, -1., 1.)
  sample = (sample + 1.) / 2. * 255
  sample = sample.numpy().astype(np.uint8)
  sample = np.transpose(sample, (0, 2, 3, 1))
  a = Image.fromarray(sample[0])
  a.save(img_out)

  if os.path.isfile(img_out):
    # display(a)
    op(c.ok, 'Upscaled image saved as', img_out.replace(drive_root, ''))
  else:
    op(c.fail, 'Error occurred: ', input.replace(drive_root, ''))

# -- END THINGS --

timer_end = time.time()

print('\nElapsed', timedelta(seconds=timer_end-timer_start))
op(c.ok, 'FIN.')
